In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy.ndimage import label
from skimage import measure 

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def greyScale(img): 
    #grayscale img 
    getRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    R, G, B = getRGB[:,:,0], getRGB[:,:,1],getRGB[:,:,2]
    grey_img = 0.2989*R + 0.5870*G + 0.1140*B
    #plt.imshow(grey_img, cmap='gray')
    #plt.show()
    return grey_img

In [3]:
def median_filter(img):
    median = cv2.medianBlur(img, 5) 
    #plt.imshow(median)
    #plt.show()
    return median

In [4]:
def threshold(img, val):
    new_img = img.copy()
    new_img[new_img <= val] = 0
    new_img[new_img > val]  = 255
    return new_img

In [5]:
def invert(img):
    inverted_img = np.copy(img)
    rows = img.shape[0]
    cols = img.shape[1]
    for i in range(rows):
        for j in range(cols):
            if img[i, j] == 0:
                inverted_img[i, j] = 1
            else:
                inverted_img[i, j] = 0    
    return inverted_img

In [6]:
def label_components(binary_img, conn):
    labeled_img, _ = label(binary_img, structure=np.ones((3, 3)))
    return labeled_img

def number_components(img, conn):
    comp_img = label_components(img, conn)
    num_components = np.max(comp_img)
    return num_components

In [7]:
def get_largest_components(img, k, conn):
    new_img = np.zeros_like(img)
    comp_img = label_components(img, conn)

    temp = np.ravel(comp_img)
    unique_labels, counts = np.unique(temp, return_counts=True)
    modes = unique_labels[np.argsort(counts)][-(k + 1):][::-1]

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if comp_img[i, j] in modes and comp_img[i, j] != 0:
                new_img[i, j] = 1

    return new_img

In [8]:
structSquare = [[-1, -1], [-1, 0], [-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1]]
structCross = [[0, 1], [0, -1], [1, 0], [-1, 0]]
structX = [[-1, -1], [-1, 1], [1, 1], [1, -1]]

In [9]:
def erode(img, struct, r):
    new_img = np.copy(img)
    orig_img = np.copy(new_img)
    for _ in range(r):
        orig_img = np.copy(new_img)
        for i in range(orig_img.shape[0]):
            for j in range(orig_img.shape[1]):
                for t in range(len(struct)):
                    if (0 <= i + struct[t][0] < orig_img.shape[0] and
                        0 <= j + struct[t][1] < orig_img.shape[1]):
                        if orig_img[i + struct[t][0], j + struct[t][1]] == 0:
                            new_img[i, j] = 0
    return new_img

def dilate(img, struct, r):
    new_img = np.copy(img)
    orig_img = np.copy(new_img)
    for _ in range(r):
        orig_img = np.copy(new_img)
        for i in range(orig_img.shape[0]):
            for j in range(orig_img.shape[1]):
                for t in range(len(struct)):
                    if (0 <= i + struct[t][0] < orig_img.shape[0] and
                        0 <= j + struct[t][1] < orig_img.shape[1]):
                        if orig_img[i + struct[t][0], j + struct[t][1]] == 1:
                            new_img[i, j] = 1
    return new_img

In [10]:
def open_img(img, struct, i):
    opened_img = np.copy(img)
    opened_img = erode(opened_img, struct, i)
    opened_img = dilate(opened_img, struct, i)
    return opened_img

def close_img(img, struct, i):
    closed_img = np.copy(img)
    closed_img = dilate(closed_img, struct, i)
    closed_img = erode(closed_img, struct, i)
    return closed_img

In [11]:
def overlay_img(tumor, original_img): 
    overlayImg = original_img.copy()
    for i in range(overlayImg.shape[0]): 
        for j in range(overlayImg.shape[1]):
            if tumor[i][j] == 0: 
                overlayImg[i][j] = overlayImg[i][j] * 0.5
            else: 
                overlayImg[i][j] = overlayImg[i][j]
                overlayImg[i][j][1] = 1
                overlayImg[i][j][2] = 1
    fig, axs = plt.subplots(1,2,figsize=(10,8))
    axs[0].imshow(original_img)
    axs[1].imshow(overlayImg)
    plt.show()
    return overlayImg, original_img

In [12]:
def tumorDetected(img, thresh): 
    original_img = img 
    #plt.imshow(original_img)
    #plt.show()
    img = median_filter(img)
    #plt.imshow(img)
    #plt.show
    grey_img = greyScale(img)
    binary_img = threshold(grey_img, thresh)
    
    src = grey_img.astype(np.uint8)
    th1 = cv2.adaptiveThreshold(src,230,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,5,2)
    adaptive_img = threshold(grey_img, th1)
    
    fig, axs = plt.subplots(1,3,figsize=(12,4))
    fig.suptitle('Binarization')
    axs[0].title.set_text('Original')
    axs[1].title.set_text('Gray Scale')
    axs[2].title.set_text('Binary: Adaptive Thresholding')
    axs[0].imshow(img)
    axs[1].imshow(grey_img, cmap='gray')
    axs[2].imshow(adaptive_img, cmap='gray')
    plt.show()
    
    
    skull_t = 30
    bin_temp = threshold(grey_img, skull_t)
    while(number_components(bin_temp, 1)<=10): 
        skull_t = skull_t + 10
        bin_temp = threshold(grey_img, skull_t)
    #plt.imshow(bin_temp, cmap="gray")
    #plt.show()
    skull_removal = get_largest_components(bin_temp, 1,1)
    #plt.imshow(skull_removal, cmap="gray")
    #plt.show()
    for row in range(grey_img.shape[0]): 
        for col in range(grey_img.shape[1]): 
            if skull_removal[row][col] == 0: 
                grey_img[row][col] = 0
    binary_img = threshold(grey_img,thresh)
    
    #enhanced_result, morph_gradient = intensity_enhancement(img)
    #enhanced = enhanced_result + morph_gradient
    #plt.imshow(enhanced)
    
    largest = get_largest_components(binary_img, 5, 1)
    inverted = invert(largest)
    #plt.imshow(inverted, cmap="gray")
    #plt.show()
    fillHoles = get_largest_components(inverted, 1, 1)
    #plt.imshow(fillHoles, cmap="gray")
    #plt.show()
    closed = close_img(fillHoles, structSquare, 1)
    #plt.imshow(closed, cmap="gray")
    #plt.show()
    invertBack = invert(closed)
    #plt.imshow(invertBack, cmap="gray")
    #plt.show()
    opened = open_img(invertBack, structSquare, 1)
    #plt.imshow(opened, cmap='gray')
    #plt.show()
    labeled = label_components(opened, 0)
    #plt.imshow(labeled, cmap="gray")
    #plt.show()
    tumor = get_largest_components(labeled, 1, 0)
    #plt.imshow(tumor,  cmap='gray')
    #plt.show()
    tumor_area = (sum(tumor[tumor == 1])/len(tumor))/8.045
    if(tumor_area > 0.4): 
        print("Tumor Detected")
        print("Tumor Size:", tumor_area, "mm^2")
        overlay, original_img = overlay_img(tumor, original_img)
    else: 
        print("No Tumor Detected")

In [13]:
def find_threshold(grey, thresh):
    binary_img = grey.copy()
    binary_img[binary_img <= thresh] = 0
    binary_img[binary_img > thresh]  = 255
    plt.imshow(binary_img, cmap="gray")

In [14]:
def runProgram(upload): 
    uploaded_file = list(upload.values())[0]
    file_name = uploaded_file['metadata']['name']
    file_path = "./"+file_name 
    with open(file_path, "wb") as fp:
        fp.write(uploaded_file['content'])
    image = cv2.imread(file_path)
    #plt.imshow(image)
    img = median_filter(image)
    grey_img = greyScale(img)
    t = interactive(find_threshold, grey=fixed(grey_img), thresh=widgets.FloatSlider(min=1, max=255.0, step=1))
    display(t)
    set_thresh = widgets.Button(description='Find Tumor',disabled=False)
    def on_button2_clicked(b): 
        tumorDetected(image,t.kwargs['thresh'])
    set_thresh.on_click(on_button2_clicked)
    display(set_thresh)

In [15]:
def on_button1_clicked(b):
    runProgram(MRI_image.value)

In [19]:
MRI_image = widgets.FileUpload(multiple=False)
button = widgets.Button(description='Set Threshold',disabled=False)
button.on_click(on_button1_clicked)
display(MRI_image, button)

FileUpload(value={}, description='Upload')

Button(description='Set Threshold', style=ButtonStyle())